<a href="https://colab.research.google.com/github/olorunfemibabalola/Bias-Detection-NLP/blob/main/MLA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# PROJECT: Explainable Depression Risk and Intent Classifier for Tweets
# UNIT: Machine Learning and Applications (Level 7)
# AUTHOR: Olorunfemi
# DESCRIPTION: A Multi-Output Machine Learning system utilizing Multimodal
# Feature Fusion and SHAP explainability to detect depression risk and intent.


LIBRARY SETUP

In [2]:
!pip install shap nltk pandas numpy scikit-learn matplotlib seaborn requests beautifulsoup4

In [3]:
import pandas as pd
import numpy as np
import re
import nltk
import shap
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, recall_score, f1_score, accuracy_score, precision_score, confusion_matrix, ConfusionMatrixDisplay


In [4]:
# Download necessary NLTK lexicons (Quietly)
nltk.download('vader_lexicon', quiet=True)
nltk.download('stopwords', quiet=True)

True

In [5]:
# Configuration for Reproducibility
RANDOM_STATE = 42
MAX_FEATURES = 1000  # For TF-IDF to keep training fast

MODULE 1: DATA ACQUISITION & SYNTHESIS (The "Safety Net")

In [6]:
def generate_synthetic_data(n_samples=500):
    """
    Generates synthetic data if real data is missing. It's like a backup dataset!
    Ensures the pipeline runs immediately for reporting/video.
    """
    print("\n⚠️  DATA STATUS: Real CSV not found. Generating SYNTHETIC DATA for demonstration...")
    print("    (This allows you to generate report graphs immediately.)")

    data = {'text': [], 'risk_label': [], 'intent': []}

    # Intent: Cry for Help (High Risk)
    helps = [
        "I can't do this anymore, please someone help.", "Planning to end it tonight.",
        "There is no hope left, I want the pain to stop.", "I feel like I'm drowning.",
        "Goodbye everyone, I can't take this life anymore."
    ]
    # Intent: Venting (Medium/High Risk)
    vents = [
        "So frustrated with everything right now.", "Why is life so exhausting?",
        "Crying in my room again, standard Tuesday.", "My anxiety is through the roof.",
        "I hate how I feel, just needed to say it."
    ]
    # Intent: Loneliness (Medium Risk)
    lonely = [
        "I haven't spoken to a human in days.", "Does anyone else feel invisible?",
        "I miss having friends who care.", "The silence is too loud.",
        "Just sitting here alone again."
    ]
    # Intent: Neutral (Low Risk)
    neutral = [
        "Watching Netflix and chilling.", "Great workout today!",
        "Anyone see the game last night?", "Coffee is life.",
        "Just finished my homework."
    ]

    for _ in range(n_samples // 4):
        data['text'].append(np.random.choice(helps))
        data['risk_label'].append(1)
        data['intent'].append("Cry for Help")

        data['text'].append(np.random.choice(vents))
        data['risk_label'].append(1)
        data['intent'].append("Venting")

        data['text'].append(np.random.choice(lonely))
        data['risk_label'].append(1)
        data['intent'].append("Loneliness")

        data['text'].append(np.random.choice(neutral))
        data['risk_label'].append(0)
        data['intent'].append("Neutral")

    return pd.DataFrame(data)


In [7]:
def load_and_prep_data():
    """
    Tries to load our main dataset ('Suicide_Detection.csv').
    If it can't find it (like, if we forgot to upload it!), it falls back to making fake data using `generate_synthetic_data`.
    """
    # Check for Raw Kaggle file
    try:
        df_raw = pd.read_csv('Suicide_Detection.csv', on_bad_lines='skip', engine='python')
        print("✅ DATA STATUS: Loaded raw 'Suicide_Detection.csv'. processing sample...")

        # Sample & Label Logic (Distinction Novelty)
        df_high = df_raw[df_raw['class'] == 'suicide'].sample(5000, random_state=RANDOM_STATE)
        df_low = df_raw[df_raw['class'] == 'non-suicide'].sample(4500, random_state=RANDOM_STATE)
        df = pd.concat([df_high, df_low]).sample(frac=1).reset_index(drop=True)

        # Map Risk
        df['risk_label'] = df['class'].map({'suicide': 1, 'non-suicide': 0})

        # Pseudo-Label Intent (Rule-Based Novelty)
        def get_intent(text):
            t = str(text).lower()
            if any(x in t for x in ['help', 'end', 'die', 'suicide', 'goodbye', 'kill', 'death', 'can\'t go on with life', 'over it', 'wish i wasn\'t here', 'i give up', 'no more', 'escape', 'fade away', 'take me away', 'done with life']): return "Cry for Help"
            if any(x in t for x in ['alone', 'lonely', 'nobody', 'friend', 'no one', 'no friend', 'isolated', 'by myself', 'solitude', 'deserted', 'forgotten', 'empty house', 'on my own', 'single', 'unaccompanied']): return "Loneliness"
            if any(x in t for x in ['tired', 'sad', 'cry', 'pain', 'hate', 'not good enough', 'it sucks', 'losing myself', 'losing', 'frustrated', 'stressed', 'annoyed', 'upset', 'angry', 'suffering', 'miserable', 'depressed', 'unhappy', 'agony']): return "Venting"
            if any(x in t for x in ['good', 'happy', 'great', 'fun', 'enjoy', 'positive', 'chill', 'relax', 'weekend', 'food', 'movie', 'book', 'game', 'learn', 'work', 'study', 'exercise', 'music', 'art', 'news', 'weather', 'sleep', 'rest']): return "Neutral"
            return "Neutral"

        df['intent'] = df.apply(lambda x: get_intent(x['text']) if x['risk_label'] == 1 else "Neutral", axis=1)

        return df[['text', 'risk_label', 'intent']]

    except FileNotFoundError:
        pass

    # 3. Fallback
    return generate_synthetic_data()

In [8]:
#load_and_prep_data().value_counts()

MODULE 2: PREPROCESSING & FEATURE ENGINEERING (Multimodal Fusion)

In [9]:
def clean_tweet(text):
    """
    This function cleans up the tweet text by removing stuff we don't need,
    like links, mentions, hashtags, and punctuation. Super important for good analysis!
    """
    text = str(text).lower()
    text = re.sub(r'http\S+', '', text)    # Remove URLs
    text = re.sub(r'@\w+', '', text)       # Remove Mentions
    text = re.sub(r'#', '', text)          # Remove Hashtag symbol
    text = re.sub(r'[^\w\s]', '', text)    # Remove punctuation
    return text.strip()


In [10]:
def multimodal_fusion(df):
    """
    This is where we get fancy! We turn the text into numbers by combining different types of features:
    1. The length of the tweet.
    2. How positive or negative the tweet sounds (sentiment).
    3. Important keywords using TF-IDF (Term Frequency-Inverse Document Frequency).
    """
    print("\n🔄 EXECUTION: Running Multimodal Feature Fusion...")

    # 1. Structural Mode (Tweet Length)
    df['text_len'] = df['text'].apply(len)

    # 2. Psychological Mode (VADER Sentiment)
    sid = SentimentIntensityAnalyzer()
    df['compound_sent'] = df['text'].apply(lambda x: sid.polarity_scores(str(x))['compound'])

    # 3. Semantic Mode (TF-IDF)
    tfidf = TfidfVectorizer(max_features=MAX_FEATURES, stop_words='english')
    tfidf_matrix = tfidf.fit_transform(df['cleaned_text'])
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf.get_feature_names_out())

    # FUSION STEP: Concatenate all features
    X = pd.concat([
        tfidf_df.reset_index(drop=True),
        df[['compound_sent', 'text_len']].reset_index(drop=True)
    ], axis=1)

    print(f"   Shape of Fused Feature Matrix: {X.shape}")
    return X, tfidf, sid


MODULE 3: PRESCRIPTIVE ENGINE (Wellness Suggestions)

In [11]:
def get_wellness_suggestion(intent_label):
    """
    After figuring out what someone's tweet means (their 'intent'),
    this function gives a helpful suggestion based on that intent.
    """
    strategies = {
        "Cry for Help": "🔴 URGENT: High distress detected. Please contact a helpline immediately. Grounding: Name 5 things you can see.",
        "Venting": "🔵 Insight: Emotional release is healthy. Suggestion: Take time off or try expressive journaling to process these feelings.",
        "Loneliness": "🟡 Insight: Isolation detected. Suggestion: Reach out to a trusted friend or join an online community.",
        "Neutral": "🟢 Status: No risk detected. Suggestion: Maintain current self-care routine.",
        "Unknown": "⚪ Suggestion: Monitor mood."
    }
    return strategies.get(intent_label, strategies['Unknown'])


MODULE 4: URL HANDLING (Twitter Scraper Fallback)

In [12]:
def get_tweet_from_url(url):
    """
    If someone gives us a Twitter URL, this function tries to grab the actual tweet text from it.
    It's a neat trick for getting data from web pages!
    """
    print(f"\n🔗 URL Detected: {url}")
    headers = {'User-Agent': 'Mozilla/5.0 (compatible; Discordbot/2.0; +https://discordapp.com)'}
    try:
        response = requests.get(url, headers=headers, timeout=5)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            meta = soup.find('meta', property='og:description')
            if meta: return meta['content'].strip('“').strip('”')
    except:
        pass

    # Graceful Fallback for Video Demo
    print("🔒 X.com API restrictions active. Please input text manually for demo:")
    return input("   Paste Tweet Text: ")



MODEL EVALUATION

In [13]:
def evaluate_model(Y_test, lr_pred, rf_pred, rf, X_test, tfidf_vectorizer, sid, le):
    """
    Evaluates model performance, generates plots, performs SHAP explainability,
    and conducts an ethical fairness check.
    """
    # 5. Metrics Calculation & Plotting: How well did they do?
    # Helper function to get recall, precision, and F1 for the risk prediction
    def get_metrics(y_true, y_pred):
        risk_true = y_true.iloc[:,0] # Actual risk labels
        risk_pred = y_pred[:,0] # Predicted risk labels
        recall = recall_score(risk_true, risk_pred)
        precision = precision_score(risk_true, risk_pred, zero_division=0)
        f1 = f1_score(risk_true, risk_pred, average='weighted') # Weighted F1 for imbalanced data
        return recall, precision, f1

    # Calculate metrics for both models
    lr_recall, lr_precision, lr_f1 = get_metrics(Y_test, lr_pred)
    rf_recall, rf_precision, rf_f1 = get_metrics(Y_test, rf_pred)

    # Print out a neat table of results
    print(f"\n📊 RESULTS TABLE (Use in Report):")
    print(f"   Model                |    Risk Recall  |   Risk Precision   | F1-Score (Weighted)")
    print(f"   ---------------------|-----------------|--------------------|--------------------")
    print(f"   Logistic Regression  | {lr_recall:.2%} | {lr_precision:.2%} | {lr_f1:.2%}")
    print(f"   Random Forest        | {rf_recall:.2%} | {rf_precision:.2%} | {rf_f1:.2%}")

    # Plot a comparison of the metrics
    plt.figure(figsize=(10,6))
    bar_width = 0.2
    index = np.arange(2)

    plt.bar(index, [lr_recall, rf_recall], bar_width, label='Recall', color='teal')
    plt.bar(index + bar_width, [lr_precision, rf_precision], bar_width, label='Precision', color='orange')
    plt.bar(index + 2*bar_width, [lr_f1, rf_f1], bar_width, label='F1-Score', color='purple')

    plt.xlabel('Model')
    plt.ylabel('Score')
    plt.title('Model Performance Comparison (Risk Label)')
    plt.xticks(index + bar_width, ['Logistic Regression', 'Random Forest'])
    plt.legend()
    plt.tight_layout()
    plt.savefig("comparison_metrics_plot.png") # Save the plot
    print("   -> Saved 'comparison_metrics_plot.png'")

    # Confusion Matrix for Random Forest (Risk Label): Visualizing correct/incorrect predictions
    print("\n📉 CONFUSION MATRIX: Generating for Random Forest (Risk Label)...")
    cm = confusion_matrix(Y_test.iloc[:,0], rf_pred[:,0]) # Calculate confusion matrix
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Low Risk', 'High Risk']) # Prepare for display
    plt.figure(figsize=(6,6))
    disp.plot(cmap=plt.cm.Blues, values_format='d') # Plot it!
    plt.title('Confusion Matrix: Random Forest (Risk Label)')
    plt.savefig("confusion_matrix_rf.png") # Save the confusion matrix plot
    print("   -> Saved 'confusion_matrix_rf.png'")

    print("\n📉 CONFUSION MATRIX: Generating for Logistic Regression (Risk Label)...")
    cm = confusion_matrix(Y_test.iloc[:,0], lr_pred[:,0]) # Calculate confusion matrix
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Low Risk', 'High Risk']) # Prepare for display
    plt.figure(figsize=(6,6))
    disp.plot(cmap=plt.cm.Blues, values_format='d') # Plot it!
    plt.title('Confusion Matrix: Random Forest (Risk Label)')
    plt.savefig("confusion_matrix_rf.png") # Save the confusion matrix plot
    print("   -> Saved 'confusion_matrix_rf.png'")

    # 6. SHAP Explainability: Why did the model make that decision?
    print("\n🔍 EXPLAINABILITY: Generating SHAP Summary Plot...")
    explainer = shap.TreeExplainer(rf.estimators_[0]) # SHAP for the Random Forest's first estimator (the risk model)

    # Sample a small part of the test set for faster SHAP calculation
    shap_sample = X_test.iloc[:50]
    # Get SHAP values, focusing on the positive class (risk_label=1)
    shap_values = explainer.shap_values(shap_sample, check_additivity=False)

    plt.figure()

    try:
        if isinstance(shap_values, list) and len(shap_values) > 1:
            # If SHAP returns values for both classes, pick the positive one
            shap.summary_plot(shap_values[1], shap_sample, show=False)
        else:
            # Otherwise, use the direct SHAP values
            shap.summary_plot(shap_values, shap_sample, show=False)

        plt.title("SHAP: Feature Impact on Depression Risk")
        plt.tight_layout()
        plt.savefig("shap_summary.png") # Save the SHAP plot
        print("   -> Saved 'shap_summary.png' ")
    except Exception as e:
        print(f"   -> SHAP summary plot could not be generated. Error: {e}")

    # 7. Ethical Fairness Check: Is our model fair across different types of tweets?
    print("\n⚖️ ETHICS CHECK: Analyzing Disparate Recall...")
    test_df = X_test.copy() # Make a copy for analysis
    test_df['pred'] = rf_pred[:, 0] # Add predictions
    test_df['true'] = Y_test.iloc[:, 0] # Add actual labels

    # Separate tweets by length to check for bias
    short = test_df[test_df['text_len'] < 50]
    long_t = test_df[test_df['text_len'] >= 50]

    # Calculate recall for short vs. long tweets
    rec_s = recall_score(short['true'], short['pred']) if len(short) > 0 else 0
    rec_l = recall_score(long_t['true'], long_t['pred']) if len(long_t) > 0 else 0

    print(f"   Recall on Short Tweets: {rec_s:.2%}")
    print(f"   Recall on Long Tweets:  {rec_l:.2%}")

MAIN PIPELINE

In [ ]:
def run_pipeline():
    # 1. Data Prep: Get our tweet data ready
    df = load_and_prep_data() # Load and clean up raw tweet data
    df['cleaned_text'] = df['text'].apply(clean_tweet) # Apply our custom cleaning function

    # 2. Features: Turn text into numbers our model can understand
    X, tfidf_vectorizer, sid = multimodal_fusion(df) # Combine length, sentiment (VADER), and keyword (TF-IDF) features

    # 3. Targets (Multi-Output Setup): What are we trying to predict?
    le = LabelEncoder() # For converting intent names to numbers
    df['intent_enc'] = le.fit_transform(df['intent']) # Encode the 'intent' column
    Y = df[['risk_label', 'intent_enc']] # Our two targets: risk (0/1) and intent (encoded number)

    # Split data into training and testing sets
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=RANDOM_STATE)

    # 4. Model Comparison: Let's see which model does better!
    print("\n🧠 TRAINING: Comparing Logistic Regression vs Random Forest Model...")

    # Baseline: Logistic Regression (a simpler model for comparison)
    lr = MultiOutputClassifier(LogisticRegression(max_iter=1000)) # Handles multiple outputs (risk & intent)
    lr.fit(X_train, Y_train)
    lr_pred = lr.predict(X_test)

    # Distinction: Random Forest (our main, more complex model)
    rf_base = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=RANDOM_STATE) # 'balanced' helps with uneven classes
    rf = MultiOutputClassifier(rf_base)
    rf.fit(X_train, Y_train)
    rf_pred = rf.predict(X_test)

    # Call the new evaluation function
    evaluate_model(Y_test, lr_pred, rf_pred, rf, X_test, tfidf_vectorizer, sid, le)

    # 8. Try it out yourself!
    print("\n" + "="*50)
    print("🚀 WE'RE LIVE!!!")
    print("="*50)

    while True: # Loop to allow multiple inputs
        user_in = input("Enter Tweet Text OR Twitter URL (type 'quit' to exit): ").strip()

        if user_in.lower() == 'quit': # Exit condition
            print("Exiting live demo. Goodbye!")
            break

        if user_in.startswith("http"): # If it's a URL, try to scrape the tweet text
            text_in = get_tweet_from_url(user_in)
        else:
            text_in = user_in # Otherwise, it's direct text input

        # Process Input for the model
        clean_in = clean_tweet(text_in) # Clean the user's input
        feat_tfidf = tfidf_vectorizer.transform([clean_in]).toarray() # Convert to TF-IDF features
        feat_sent = sid.polarity_scores(clean_in)['compound'] # Get sentiment score
        feat_len = len(clean_in) # Get text length

        # Combine all features into one array
        feat_full = np.hstack([feat_tfidf, [[feat_sent, feat_len]]])

        # Predict risk and intent using the Logistic Regression model (can be changed to RF if preferred)
        pred_raw = lr.predict(feat_full)
        try:
            intent_res = le.inverse_transform([pred_raw[0][1]])[0] # Decode intent label
        except:
            intent_res = "Unknown" # Fallback if decoding fails

        # Derive granular risk level based on predicted intent
        if intent_res == "Cry for Help":
            risk_res = "High Risk"
        elif intent_res == "Venting":
            risk_res = "Moderate Risk"
        elif intent_res == "Loneliness":
            risk_res = "Low Risk"
        elif intent_res == "Neutral":
            risk_res = "No Risk"
        else:
            risk_res = "Uncertain Risk" # For 'Unknown' or other unexpected intents

        # Display the analysis and a wellness suggestion
        print(f"\n📝 Analysis for: '{text_in}'")
        print(f"   Risk Level:  {risk_res}")
        print(f"   Intent:      {intent_res}")
        print(f"   Suggestion:  {get_wellness_suggestion(intent_res)}")
        print("="*50)

In [14]:
if __name__ == "__main__":
    run_pipeline()

✅ DATA STATUS: Loaded raw 'Suicide_Detection.csv'. processing sample...

🔄 EXECUTION: Running Multimodal Feature Fusion...
   Shape of Fused Feature Matrix: (9500, 1002)

🧠 TRAINING: Comparing Logistic Regression vs Random Forest Model...


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



🚀 LIVE DEMO SYSTEM (Ready for Video Recording)
Enter Tweet Text OR Twitter URL (type 'quit' to exit): I want to eat so much ice cream today!


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(



📝 Analysis for: 'I want to eat so much ice cream today!'
   Risk Level:  No Risk
   Intent:      Neutral
   Suggestion:  🟢 Status: No risk detected. Suggestion: Maintain current self-care routine.
Enter Tweet Text OR Twitter URL (type 'quit' to exit): quit
Exiting live demo. Goodbye!
